In [2]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, select, TryCast, MetaData
from abc import ABC, abstractmethod
import json
import pyodbc
import pandas as pd
import requests


with open("./data.json", 'r+') as file:
    data = json.load(file)
print(data, type(data))
print(data['username'])


conn_url = URL.create(
    "mssql+pyodbc",
    username=data['username'],
    password=data["password"],
    host=data["host"],
    port=1433,
    database=data["database"],
    query={
        "driver": "ODBC Driver 18 for SQL Server",
        "TrustServerCertificate": "yes"
    }
)
engine = create_engine(conn_url)


dataframe = pd.read_sql_query("select * from ONE_Bank_Data", engine)
dataframe

{'username': 'KPI-DB-User_reporter', 'password': '3NQ7ySjru6IixLGf/zU2/', 'host': 'kpi-db-server.database.windows.net', 'port': 1433, 'database': 'kpi-db', 'query': {'driver': 'ODBC Driver 18 for SQL Server', 'TrustServerCertificate': 'yes', 'authentication': 'ActiveDirectoryIntegrated'}} <class 'dict'>
KPI-DB-User_reporter


,TopsAgentBankID,CustomerID,AccountName,Routing,StatementBalance
0,103,4448,AAB MM Reserve 8268,122105980,10192.63
1,104,4448,Alliance Association Bank OP,122105980,151991.69
2,105,4448,Pacific Western Checking 4807,122238200,0.00
3,106,4448,Alliance Association Bank OP,122105980,91090.93
4,107,4448,RBC Wealth Management,777777777,551427.49
...,...,...,...,...,...
84538,267,4679,Davis Pond Reserves,111025453,0.00
84539,268,4679,Foxcreek Operating Account,111025453,0.00
84540,269,4679,Foxcreek Reserves,111025453,0.00
84541,270,4679,Foushee Mews Operating,111025453,0.00


In [4]:
dataset = dataframe.iloc[:, 3].values

banks = {}
try:
    for i in dataset:
        url = f'https://www.routingnumbers.info/api/name.json?rn={i}'
        response = requests.get(url)
        data = json.loads(response.content)
        if data['message'] == 'OK':
            banks[i] = data['name']
        else:
            pass
except:
    pass

routing = []
bank_name = []
for i in banks:
    routing.append(i)
for i in banks.values():
    bank_name.append(i)
new_dataframe = pd.DataFrame(list(zip(bank_name, routing)), columns=[
                            'Bank', 'Routing'])

df1 = dataframe.merge(new_dataframe, on='Routing')